In [4]:
# library(dplyr)

In [2]:
metadata_df = read.csv('/nfs/leia/research/stegle/dseaton/hipsci/singlecell_neuroseq/data/metadata/All_samples_list_CGaP_170628_overview_processed.txt',
                         sep='\t')

head(metadata_df)

Public.name,Donor,Gender,Age,feederfree
HPSI0813er-fpdk,fpdk,male,40-44,
HPSI0813i-fpdk_1,fpdk,male,40-44,n
HPSI0813i-fpdk_2,fpdk,male,40-44,n
HPSI0813i-fpdk_3,fpdk,male,40-44,n
HPSI0813er-fpdr,fpdr,male,45-49,
HPSI0813i-fpdr_1,fpdr,male,45-49,n


In [18]:
metadata_df[grep("mioj",metadata_df$Public.name),]
metadata_df[grep("oesx",metadata_df$Public.name),]

,Public.name,Donor,Gender,Age,feederfree
1542,HPSI0813pf-mioj,mioj,,,n
1543,HPSI0813i-mioj_1,mioj,,,y
1544,HPSI0813i-mioj_4,mioj,,,y


,Public.name,Donor,Gender,Age,feederfree
1545,HPSI0613pf-oesx,oesx,,,n
1546,HPSI0613i-oesx_3,oesx,,,y
1547,HPSI0613i-oesx_5,oesx,,,y


In [3]:
nrow(metadata_df)

[1] 2011

In [10]:
# coldata = read.csv("/hps/nobackup/stegle/users/acuomo/all_scripts/sc_neuroseq/coldata.csv")
# head(coldata)
donors = unique(coldata$donor_id)
length(donors)

[1] 213

In [21]:
donors[!(donors %in% metadata_df$Public.name)]
# sohd:female, uict:female, qecv:female, pitg:female 

[1] HPSI0514i-sohd_2 HPSI0314i-uict_1 HPSI0115i-qecv_2 HPSI0115i-pitg_2
213 Levels: HPSI0114i-bezi_3 HPSI0114i-eipl_1 ... HPSI1213i-tolg_4

In [20]:
meta = metadata_df[metadata_df$Public.name %in% donors,]
nrow(meta)
meta[meta$Donor == "oesx","Gender"] <- "female"
meta[meta$Donor == "mioj","Gender"] <- "female"
nrow(meta[meta$Gender == 'male',])
nrow(meta[meta$Gender == 'female',])

[1] 209

[1] 86

[1] 123

In [23]:
passage_df = read.csv('/hps/nobackup/stegle/users/acuomo/all_scripts/sc_neuroseq/diff_efficiency/CellLine_efficiency&Passages.csv')
head(passage_df)

CellLine,Pool,P.Thaw,P.Pool,X.Pthaw_Pool,Harvest_Confluency,diff_efficiency,category
paab_3,18,22,26,4,50,0.00000000,bad
paab_3,5,22,25,3,NA,0.00000000,bad
uofv_1,13,53,55,2,NA,0.00000000,bad
mita_1,6,53,56,3,NA,0.01192354,bad
eipl_1,1,38,42,4,80,0.01287835,bad
boss_1,13,26,29,3,NA,0.01419878,bad


In [24]:
passage_df = passage_df %>% group_by(CellLine) %>% summarise(avg_p_thaw = mean(P.Thaw),
                                               avg_p_pool = mean(P.Pool))
passage_df$short_donor_id = passage_df$CellLine
passage_df$CellLine <- c()
head(passage_df)

avg_p_thaw,avg_p_pool,short_donor_id
6,9,$ASXL3_HM2
6,10,$CHD2_HT1
6,10,$CHD7_HT1
5,8,$CTNNB1_HT1
5,9,$GATAD2B_HT1
9,14,$KMT2E_HT3


In [5]:
diff_df = read.csv('/nfs/leia/research/stegle/dseaton/hipsci/singlecell_neuroseq/data/data_processed/pool1_17_D52/pool1_17_D52.scanpy.w_metadata.w_celltype.scanpy.obs_df.groupedby.donor_id-pool_id-time_point.diff_efficiency_by_cell_line.tsv',
                     sep='\t')
diff_df$short_donor_id = gsub(".*i-","",diff_df$donor_id)
head(diff_df)
nrow(diff_df)

donor_id,diff_efficiency,short_donor_id
HPSI0114i-bezi_3,0.58269720,bezi_3
HPSI0114i-eipl_1,0.01287835,eipl_1
HPSI0114i-fikt_3,0.40282686,fikt_3
HPSI0114i-kolf_2,0.47059133,kolf_2
HPSI0114i-lexy_2,0.72690535,lexy_2
HPSI0114i-posc_1,0.81349206,posc_1


[1] 199

In [26]:
df0 = inner_join(passage_df,diff_df)
nrow(diff_df)
nrow(df0)

Joining, by = "short_donor_id"
Warning message:
“Column `short_donor_id` joining factor and character vector, coercing into character vector”

[1] 199

[1] 199

In [27]:
nrow(diff_df)
nrow(diff_df[diff_df$diff_efficiency < 0.2,])
nrow(diff_df[diff_df$diff_efficiency < 0.2,])/nrow(diff_df)

[1] 199

[1] 52

[1] 0.2613065

In [28]:
metadata_df$donor_id = metadata_df$Public.name

In [29]:
# df = inner_join(diff_df, metadata_df)
df = inner_join(df0, metadata_df)
nrow(df)
head(df)

Joining, by = "donor_id"
Warning message:
“Column `donor_id` joining factors with different levels, coercing to character vector”

[1] 195

avg_p_thaw,avg_p_pool,short_donor_id,donor_id,diff_efficiency,Public.name,Donor,Gender,Age,feederfree
35,37,aehn_22,HPSI0713i-aehn_22,0.9211045,HPSI0713i-aehn_22,aehn,female,,y
33,36,aevs_1,HPSI0613i-aevs_1,0.1051095,HPSI0613i-aevs_1,aevs,male,,n
34,36,aion_2,HPSI0115i-aion_2,0.7118205,HPSI0115i-aion_2,aion,male,60-64,y
47,50,aipt_33,HPSI0513i-aipt_33,0.7129151,HPSI0513i-aipt_33,aipt,male,,y
24,27,aizi_3,HPSI0813i-aizi_3,0.9523810,HPSI0813i-aizi_3,aizi,female,50-54,n
15,18,aowh_2,HPSI0715i-aowh_2,0.6208233,HPSI0715i-aowh_2,aowh,female,65-69,y


In [32]:
fit = lm(formula = diff_efficiency ~ Gender + Age + feederfree + avg_p_thaw + avg_p_pool, data = df)
summary(fit)
anova(fit)


Call:
lm(formula = diff_efficiency ~ Gender + Age + feederfree + avg_p_thaw + 
    avg_p_pool, data = df)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.73037 -0.20126  0.02298  0.22902  0.45906 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)   0.57609    0.30958   1.861 0.064448 .  
Genderfemale  0.22887    0.29385   0.779 0.437116    
Gendermale    0.11775    0.29436   0.400 0.689631    
Age25-29      0.01360    0.17599   0.077 0.938504    
Age30-34     -0.25417    0.20993  -1.211 0.227640    
Age35-39     -0.13780    0.12939  -1.065 0.288327    
Age40-44     -0.43129    0.13171  -3.275 0.001277 ** 
Age45-49     -0.43092    0.11550  -3.731 0.000258 ***
Age50-54     -0.23699    0.09107  -2.602 0.010055 *  
Age55-59     -0.15455    0.08236  -1.877 0.062260 .  
Age60-64     -0.21629    0.08133  -2.659 0.008561 ** 
Age65-69     -0.25167    0.08763  -2.872 0.004588 ** 
Age70-74     -0.15117    0.09502  -1.591 0.113433    
Age75-79     -0.24

,Df,Sum Sq,Mean Sq,F value,Pr(>F)
Gender,2,0.6161299,0.30806493,3.824696,0.023682704
Age,11,2.1945223,0.19950203,2.476863,0.006591788
feederfree,1,0.2703243,0.27032429,3.356137,0.068664642
avg_p_thaw,1,0.2913396,0.29133956,3.617046,0.058842601
avg_p_pool,1,0.1286144,0.12861442,1.596777,0.208050949
Residuals,174,14.0150499,0.08054626,NA,NA
